In [ ]:
%matplotlib inline

### Load config

In [ ]:
from config import batch_size, train_split, get_path_and_names
data_dir, model_path, quant_model_path, debug_class = get_path_and_names()


In [ ]:
from loader import load_split_dataset


### Comprueba la carpeta con el dataset

In [ ]:
from os import listdir
import cv2
import random
from output import show_image


In [ ]:
input_labels = listdir(data_dir)
input_labels[:5]

In [ ]:
label = input_labels[random.randint(0, len(input_labels) - 1)]
label_images = listdir(data_dir + "/" + label)
label_images

print(label)

image = cv2.imread(data_dir + "/" + label + "/" + label_images[0])
show_image(image)


### Carga el dataset

In [ ]:
train_loader, test_loader, classes = load_split_dataset(
    data_dir,
    train_split=train_split,
    batch_size=batch_size,
)

num_classes = len(classes)

print(len(train_loader))
print(len(test_loader))
print(train_loader.batch_size)


### Comprueba si las clases etsán balanceadas entre train y test

In [ ]:
from utils import check_balance_status

In [ ]:
def check_balance():
    train_class_count = check_balance_status(train_loader, classes)
    test_class_count = check_balance_status(test_loader, classes)

    pc_mean = 0

    for tr, te in zip(train_class_count, test_class_count):
        tr_pc = tr[1] / (tr[1] + te[1]) * 100
        pc_mean += tr_pc
        print(f"{tr[0]} - TR: {tr[1]} TS: {te[1]} - PC-TR: {tr_pc:.2f}%")

    pc_mean /= len(train_class_count)
    print(f"Mean PC-TR: {pc_mean:.2f}%")


# check_balance()


In [ ]:
from model import create_model, change_last_layer, export_model


### Creación del modelo para entrenar

In [ ]:
model = create_model()
change_last_layer(model, num_classes)


### Entrenamiento

In [ ]:
from output import plot_costs
from train import train_model

In [ ]:
costs = train_model(model, train_loader)
plot_costs(costs)


### Test

In [ ]:
from test import check_accuracy, predict_class

In [ ]:
print("Checking accuracy on Training Set and normal model")
check_accuracy(train_loader, model, classes)


In [ ]:
print("Checking accuracy on Test Set and normal model")
check_accuracy(test_loader, model, classes)

### Check train and test on debug class.

In [ ]:
print(f"Prediction for {debug_class} and train set")
predict_class(train_loader, model, classes, debug_class)


In [ ]:
print(f"Prediction for {debug_class} and test set")
predict_class(test_loader, model, classes, debug_class)


### Save model to further use

In [ ]:
export_model(model, model_path)
